In [1]:
#### import tatqa_utils
import pandas as pd
import table_convert
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2


In [2]:
import os
with open('dataset_raw/openai.api.key.2', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [3]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_community.cache import SQLiteCache

#llm = ChatOpenAI(temperature=0)
#llm = OpenAI(temperature=0) 
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [4]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [5]:
def get_question(qid):
    for i, item in devdf.iterrows():
        for q in item['questions']:        
            if q['uid'] == qid:
                #table = item['table']['table']
                return (item['table'], q)
    return (None, None)

In [6]:
def gen_code3(llm, messages, question, value_list): 
    
    prompt = ChatPromptTemplate.from_messages(messages)

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser
    
    response = chain.invoke({"value_list": value_list, "question":question})
    code =  response.replace('```python','').replace('```','')
    
    return (prompt.format(value_list = value_list, question = question), code)

def exec_code(code, value_list):  
        try: 
            loc = locals()   
            if not "run()" in code:
                exec(code + f"\nr = run({value_list})\n", globals(), loc)
            else: 
                exec(code + "\nr = run()\n", globals(), loc)
            return loc['r']
        except Exception as e:
                s = '[Error]'+ str(e)
                print(s)
                return (s,'')

def gen_code_table(llm, messages, question, table): 
    
    prompt = ChatPromptTemplate.from_messages(messages)

    output_parser = StrOutputParser()

    chain = prompt | llm | output_parser
    
    response = chain.invoke({"table": table, "question":question})
    code =  response.replace('```python','').replace('```','')
    
    return (prompt.format(table = table, question = question), code)

def exec_code_table(code, table):  
        try: 
            loc = locals()   
            if not "run()" in code:
                exec(code + f"\nr = run({table})\n", globals(), loc)
            else: 
                exec(code + "\nr = run()\n", globals(), loc)
            return loc['r']
        except Exception as e:
                s = '[Error]'+ str(e)
                print(s)
                return (s,'')

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0, top_p = 1.0, seed=11)
llm = ChatOpenAI(model="gpt-3.5-turbo", 
temperature=0,
    #max_tokens=150,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    seed=123)

In [12]:
messages=[
         ("system","You are a helpful assistant with a subtask in a question-answering pipeline. The questions are related to a financial report. " + 
             "Financial report is stored as a list of annotated values. You will receive the financial report as an annotated value list and the question. "+
             "Your task is to generate a Python function that can calculate a numeric value that is the answer for the received question. "                            
         ),
        (
          "human",
          "Here is the financial report as a list of annotated values: {value_list}"
        ),         
        (
          "ai",
          "Ok, I received the value list."
        ),
        (
          "human",
          "Here is the question requires calculation on the financial report: {question}"
        ),
        (
          "ai",
          "Ok, I received the question."
        ),
        (
          "human",
            "Generate a Python function 'run(value_list)' that can answer the question using the list of annotated values! "+
            "The function must return a tuple (number, scale). The resulting number is a float with accuracy to two decimal places. Scale usually is thousand, million, billion, percent or an empty string. "+
            "The calculation usually involves two steps: a selection and a calculation on selected data. "+
            "If the question is about calculating the year average, you must calculate the average between the given year and the previous one. ex. 2015_average = (2015_value + 2014_value)/2  "+
            "If the question is about calculating the change between year averages, apply the previous logic and take difference. " +            
            "Expenses are revenue minus net income. " +
            "To calculate the difference, use absolute value. " +
            "To calculate proportion, do not calculate percentage. " +
            "Use all given year values if no year specified. " +
            "The code must be specific to the provided value list. " +
            "Do not generate explanation, nor example code, just the function. "                  
        ),
        (
          "ai",
          "Ok, I have all the information. The Python function is as follows:"
        ),
      ]     

messages_table=[
         ("system","You are a helpful assistant with a subtask in a question-answering pipeline. The questions are related to a financial report. " + 
             "Financial report is stored as a table. You will receive the financial report as a table and the question. "+
             "Your task is to generate a Python function that can calculate a numeric value that is the answer for the received question. "                            
         ),
        (
          "human",
          "Here is the financial report as a table: {table}"
        ),         
        (
          "ai",
          "Ok, I received the value list."
        ),
        (
          "human",
          "Here is the question requires calculation on the financial report: {question}"
        ),
        (
          "ai",
          "Ok, I received the question."
        ),
        (
          "human",
            "Generate a Python function 'run(table)' that can answer the question using the table! "+
            "The function must return a tuple (number, scale). The resulting number is a float with accuracy to two decimal places. Scale usually is thousand, million, billion, percent or an empty string. "+
            "The calculation usually involves two steps: a selection and a calculation on selected data. "+ 
            "To select a row match the first cell value in code! "+
            "If the question is about calculating the year average, you must calculate the average between the given year and the previous one. ex. 2015_average = (2015_value + 2014_value)/2  "+
            "If the question is about calculating the change between year averages, apply the previous logic and take difference. " +            
            "Expenses are revenue minus net income. " +
            "To calculate the difference, use absolute value. " +
            "To calculate proportion, do not calculate percentage. " +
            "Use all given year values if no year specified. " +
            "The code must be specific to the provided table. " +
            "Do not generate explanation, nor example code, just the function."
        ),
        (
          "ai",
          "Ok, I have all the information. The Python function is as follows:"
        ),
      ] 

In [13]:
#['Period', 'Total Number of Shares Purchased', 'Average Price Paid per Share', 'Total Number of Shares Purchased as Part of Publicly Announced Program', 'Approximate Dollar Value of Shares That May Yet Be Purchased Under The Repurchased Program']
#['', '(Shares in thousands)', '', '(Shares in thousands)', '(Dollars in millions)']
#['January 26, 2019 - February 22, 2019', '262', '$ 64.77', '306,255', '$ 2,372']
#'February 23, 2019 - March 22, 2019', '3,380', '$ 65.53', '309,635', '$ 2,150']
#'March 23, 2019 - April 26, 2019', '3,608', '$72.49', '313,244', '$ 1,889']
#'Total', '7,250', '$68.97', '', '']

values = [{'number_value': 262.0, 'scale': '', 'category': 'January 26, 2019 - February 22, 2019', 'header1': '', 'header2': 'Total Number of  Shares Purchased'},
{'number_value': 64.77, 'scale': '', 'category': 'January 26, 2019 - February 22, 2019', 'header1': 'Average Price Paid per Share'},
{'number_value': 306255.0, 'scale': '', 'category': 'January 26, 2019 - February 22, 2019', 'header1': '(Shares in thousands)', 'header2': 'Total Number of Shares Purchased as Part of Publicly Announced Program'},
{'number_value': 2372.0, 'scale': '', 'category': 'January 26, 2019 - February 22, 2019', 'header1': '(Dollars in millions)', 'header2': 'Approximate Dollar Value of Shares That May Yet Be Purchased Under The Repurchased Program'},
{'number_value': 3380.0, 'scale': '', 'category': 'February 23, 2019 - March 22, 2019', 'header1': '', 'header2': 'Total Number of Shares Purchased'},
{'number_value': 65.53, 'scale': '', 'category': 'February 23, 2019 - March 22, 2019', 'header1': 'Average Price Paid per Share'},
{'number_value': 309635.0, 'scale': '', 'category': 'February 23, 2019 - March 22, 2019', 'header1': '(Shares in thousands)', 'header2': 'Total Number of Shares Purchased as Part of Publicly Announced Program'},
{'number_value': 2150.0, 'scale': '', 'category': 'February 23, 2019 - March 22, 2019', 'header1': '(Dollars in millions)', 'header2': 'Approximate Dollar Value of Shares That May Yet Be Purchased Under The Repurchased Program'},
{'number_value': 3608.0, 'scale': '', 'category': 'March 23, 2019 - April 26, 2019', 'header1': '', 'header2': 'Total Number of Shares Purchased'},
{'number_value': 72.49, 'scale': '', 'category': 'March 23, 2019 - April 26, 2019', 'header1': 'Average Price Paid per Share'},
{'number_value': 313244.0, 'scale': '', 'category': 'March 23, 2019 - April 26, 2019', 'header1': '(Shares in thousands)', 'header2': 'Total Number of Shares Purchased as Part of Publicly Announced Program'},
{'number_value': 1889.0, 'scale': '', 'category': 'March 23, 2019 - April 26, 2019', 'header1': '(Dollars in millions)', 'header2': 'Approximate Dollar Value of Shares That May Yet Be Purchased Under The Repurchased Program'},
{'number_value': 7250.0, 'scale': '', 'category': 'Total', 'header1': '', 'header2': 'Total Number of Shares Purchased'},
{'number_value': 68.97, 'scale': '', 'category': 'Total', 'header1': 'Average Price Paid per Share'}]

In [15]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

qid = 'ba6783f3-8207-419a-b407-3f688682caef'
qid = 'a0414f81-8dc2-44b2-a441-2c9d9c805c4d'

#qid = 'bf7abd62-d9cd-48d2-8826-1457684019a3'
#qid = '4d259081-6da6-44bd-8830-e4de0031744c'
#qid = 'dc5e217a-a7b3-4fc9-ac0f-13d328f26b20'

#qid = '7cd3aedf-1291-4fea-bc9d-a25c65727b7b'
#qid = '22e20f25-669a-46b9-8779-2768ba391955' # 547 year avreage change 
#qid = '65ec782c-691e-45df-b541-caecb85154ff'
#qid = 'a983501d-2eec-486d-9661-e520c7c8af5e'
#qid = 'a81f1322-e74f-4e3c-a6cf-4b8d25d01cf5'
#qid = '64c902c6-f426-4432-84b3-c10b3065716f'
#qid = '6100c476-160a-4f1e-bfc1-a16f4cc18b52'
#qid = 'ef274d2b-fbd6-4e9f-95f9-79b37827d91c'
#qid = '94916f79-4b6d-487f-8fa2-39f81a085760'
#qid = '72325ec6-41ad-4648-9798-b22a61122cb4'
#qid = '263d03ec-83d2-48df-8376-1a72167798f7'
#qid = '4f7c8e0a-6ae9-40ee-bd16-06db6f96eaf1'
#qid = '35d602ae-9131-4291-a30c-49a40f32bbe4'

#qid = 'bd2d81eb-46fc-4e62-908d-aebfccf46246' # felreertheto sort is bevesz
#qid = '01de2123-400f-4411-98d9-cd3f9f1e4136' # szovegben is volt fontos ertek
#qid = 'fec503e8-6f91-483e-856d-bb1278bd031f' # zarojel problema
#qid = '59f13f19-4494-4c54-8e9a-1b499d50021a'
#qid = 'f12806ca-aa4c-4fe6-84fc-10cefca76a9e'
#qid = 'f36e5912-f63c-4837-a9ed-ddbe73e5148b'
#qid = 'eb50d3aa-16da-4087-9ab2-10106ebd10e7'
#qid = '8b2d5db2-7a47-4269-9df2-f44385bc0883'

#tablazat javitas utan megjelentek
#qid = '010df393-a62e-408d-888a-045a4e435e6a'
#qid = '73693527-ed4b-4d07-941e-0e654095a43d'
#qid = '426e47ae-aff7-43e2-81be-0bb951b0def1'
#qid = 'b8afd327-b836-4276-8556-eac89746b09f'
#qid = '6ba4045d-4222-4da0-a126-94b267a707ec'
#qid = 'e9b75381-50b1-4717-a322-31ed1eef4259'
#qid = 'd06a5ade-d848-4325-a2a5-8f5ef427d246'
#qid = '3502f875-f816-4a00-986c-fef9b08c0f96'

#qid = '99f83ba7-2e57-44d8-ac0a-64ab3e9c7b47' # subheader -t nem vette figyelembe

#Error
#qid = '5c8c999e-354f-4693-9b2d-29e3c03cb2af' # vonast nullanak kell ertelmezni? -
#qid = 'b382a11b-749b-425a-a77d-20e943e00f77' # +
#qid = 'e18e3335-7e7a-4138-9f1b-813604e44c3c'
#qid = 'e02b2c21-3972-41cc-b391-5478da7df9d8'
_table, _q =  get_question(qid)
table = _table['table']
print("tid: ",_table['uid'])
[print(r) for r in table]

q = _q['question']
print(q)
values = table_convert.convert_multitable(table)


[print(v) for v in values]
#response = chain.invoke({"value_list": values, "question":q})
#code =  response.replace('```python','').replace('```','')
p, code = gen_code3(llm, messages, q, values)
#p, code = gen_code_table(llm, messages_table, q, table)
print(code)
#print(len(p) + len(code))

(v, s) = exec_code(code, values)
#(v, s) = exec_code_table(code, table)

#prompt_template = ChatPromptTemplate.from_messages(messages)
#prompt = prompt_template.format(dataset=values, question=q)

#prompt = ChatPromptTemplate.from_messages(messages_direct)

#output_parser = StrOutputParser()

#chain = prompt | llm | output_parser
    
#response = chain.invoke({"dataset": values, "question":q})
#print(response)
#code =  response.replace('```python','').replace('```','')
#(v, s) = exec_code(code, values)


rng = 0.01
if isinstance(v, float) and ( v == _q['answer'] or (v > _q['answer']-rng and v < _q['answer']+rng)):
    print("\033[92m Success: " + str(v)+'\033[0m')
else:    
    print("\033[91m failure: " + str(v), 'good answer: ', _q['answer'],'\033[0m' )  

tid:  6bf238a5-0a3e-492d-91f8-7f62d3b37fba
['Income statement expense', '', '', '']
['', '2019 €m', '2018 €m', '2017 €m']
['Defined contribution schemes', '166', '178', '192']
['Defined benefit schemes', '57', '44', '20']
['Total amount charged to income statement (note 23)', '223', '222', '212']
What is the 2019 average defined contribution schemes?
{'number_value': 166.0, 'scale': 'million', 'category': 'Defined contribution schemes', 'header1': '2019', 'header2': 'Income statement expense'}
{'number_value': 178.0, 'scale': 'million', 'category': 'Defined contribution schemes', 'header1': '2018', 'header2': 'Income statement expense'}
{'number_value': 192.0, 'scale': 'million', 'category': 'Defined contribution schemes', 'header1': '2017', 'header2': 'Income statement expense'}
{'number_value': 57.0, 'scale': 'million', 'category': 'Defined benefit schemes', 'header1': '2019', 'header2': 'Income statement expense'}
{'number_value': 44.0, 'scale': 'million', 'category': 'Defined benef

In [16]:
pd.DataFrame(table)

,0,1,2,3
0,Income statement expense,,,
1,,2019 €m,2018 €m,2017 €m
2,Defined contribution schemes,166,178,192
3,Defined benefit schemes,57,44,20
4,Total amount charged to income statement (note...,223,222,212


In [17]:
pd.DataFrame(values)

,number_value,scale,category,header1,header2
0,166.0,million,Defined contribution schemes,2019,Income statement expense
1,178.0,million,Defined contribution schemes,2018,Income statement expense
2,192.0,million,Defined contribution schemes,2017,Income statement expense
3,57.0,million,Defined benefit schemes,2019,Income statement expense
4,44.0,million,Defined benefit schemes,2018,Income statement expense
5,20.0,million,Defined benefit schemes,2017,Income statement expense
6,223.0,million,Total amount charged to income statement (note...,2019,Income statement expense
7,222.0,million,Total amount charged to income statement (note...,2018,Income statement expense
8,212.0,million,Total amount charged to income statement (note...,2017,Income statement expense


In [93]:
#negative
qids = ['ef9d4839-5277-4614-bb73-f902fd8a38b6',
'147f7c09-a627-4719-93ad-33661b5a4543',
'f1034ee7-b182-48d5-bad8-1d9902500f57', 
'c36e2211-e46a-43d1-a0a8-ae87af347ae8', 
'88795fae-3c5b-48d4-ae2d-57fe04b15c31', 
'ed9e3e80-6941-4888-9e40-bbd1cb092485', 
'b194279c-16e5-4a17-9733-c8babb6e8281', 
'27bc5656-93b7-4109-b1d2-25710ca3240f', 
'e314bfab-8cd6-4bb3-b0e6-df664300c380', 
'0b835494-2c10-4f6b-b54a-60aa73ebdabe', 
'9a11777d-c146-4a13-aaa0-7676223a7849', 
'f49252e2-158b-4a79-ac56-d21ea67082a8', 
'c91ad938-6ad1-4481-8f6b-43ccd2f69926', 
'524463b6-b8d7-410d-926a-699f7b7cdd99', 
'4082086c-6f47-40b1-8ac1-757d437a502d'] 

a983501d-2eec-486d-9661-e520c7c8af5e 
64c902c6-f426-4432-84b3-c10b3065716f 
010df393-a62e-408d-888a-045a4e435e6a 
147f7c09-a627-4719-93ad-33661b5a4543 *
f1034ee7-b182-48d5-bad8-1d9902500f57 *
732c81f8-a16d-4d34-9917-fa98c195feec 
0b7463b3-ed9e-47a0-b838-b26e0ab886eb 
fd59c14d-0312-4725-92d8-5c30cd5c5c7a 
0b835494-2c10-4f6b-b54a-60aa73ebdabe *
c91ad938-6ad1-4481-8f6b-43ccd2f69926 *
bdcabe8e-6738-47d3-8412-40e5e9a359d0 
524463b6-b8d7-410d-926a-699f7b7cdd99 *
4082086c-6f47-40b1-8ac1-757d437a502d *
e4f21617-1a84-40e4-bdf0-671df88cb862 
359f9c4c-b405-40e5-88b5-86bfa0069b89 


for qid in qids:
    _table, _q =  get_question(qid)
    table = _table['table']
    #print("tid: ",_table['uid'])
    #[print(r) for r in table]

    q = _q['question']
    #print(q)
    print(_q['uid'])
    values = table_convert.convert_multitable(table)
    #[print(v) for v in values]
    p, code = gen_code3(llm, messages, q, values)
    #print(len(p) + len(code))
    (v, s) = exec_code(code, values)
    rng = 0.01
    if isinstance(v, float) and ( v == _q['answer'] or (v > _q['answer']-rng and v < _q['answer']+rng)):
        print("\033[92m Success: " + str(v)+'\033[0m')
    else:    
        print("\033[91m failure: " + str(v), 'good answer: ', _q['answer'],'\033[0m' )   


SyntaxError: invalid decimal literal (464286186.py, line 18)

In [49]:
qids = ['010df393-a62e-408d-888a-045a4e435e6a',
'73693527-ed4b-4d07-941e-0e654095a43d',
'426e47ae-aff7-43e2-81be-0bb951b0def1',
'b8afd327-b836-4276-8556-eac89746b09f',
'6ba4045d-4222-4da0-a126-94b267a707ec',
'e9b75381-50b1-4717-a322-31ed1eef4259']
for qid in qids:
    _table, _q =  get_question(qid)
    table = _table['table']
    #print("tid: ",_table['uid'])
    #[print(r) for r in table]

    q = _q['question']
    #print(q)
    print(_q['uid'])
    values = table_convert.convert_multitable(table)
    #[print(v) for v in values]
    p, code = gen_code3(llm, messages, q, values)
    #print(len(p) + len(code))
    (v, s) = exec_code(code, values)
    rng = 0.01
    if isinstance(v, float) and ( v == _q['answer'] or (v > _q['answer']-rng and v < _q['answer']+rng)):
        print("\033[92m Success: " + str(v)+'\033[0m')
    else:    
        print("\033[91m failure: " + str(v), 'good answer: ', _q['answer'],'\033[0m' )   

010df393-a62e-408d-888a-045a4e435e6a
 failure: -14212.0 good answer:  14212 
73693527-ed4b-4d07-941e-0e654095a43d
 failure: 0.02 good answer:  0.9500000000000001 
426e47ae-aff7-43e2-81be-0bb951b0def1
 failure: 259.8 good answer:  112.5 
b8afd327-b836-4276-8556-eac89746b09f
[Error]'[' was never closed (<string>, line 10)
 failure: [Error]'[' was never closed (<string>, line 10) good answer:  -6841 
6ba4045d-4222-4da0-a126-94b267a707ec
[Error]
 failure: [Error] good answer:  21.33 
e9b75381-50b1-4717-a322-31ed1eef4259
 failure: -1.0 good answer:  -2 


In [18]:
def get_answer(llm, messages, table, q):
    values = table_convert.convert_multitable(table)    
    p, code = gen_code3(llm, messages, q, values)    
    #print(code)
    (v, s) = exec_code(code, values)
    return  (v, s)

In [30]:
from tqdm.notebook import tqdm as log_progress

devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')
cnt = 0
res = []
for i, item in log_progress(devdf.iterrows()):
    try:
        #if cnt == 36:
        #    break;
        #print(item['table']['uid'])
        table = item['table']['table']
        value_list = table_convert.convert_multitable(table)    
        
        for q in item['questions']:        
            if q['answer_type'] == 'arithmetic' and 'table' == q['answer_from']:   
                cnt = cnt + 1
                print(q['uid'])
                _table, _q =  get_question(q['uid'])
                
                table = _table['table']
                
                #[print(r) for r in table]
                
                q = _q['question']
                print(q, end='')
                
                
                r = []
                #for i in [0,1,2]:        
                #   r.append(get_answer(llm, messages, table, q))
                
                #if r[0][0] == r[1][0]:
                #    (v, s) = r[0]
                #elif r[1][0] == r[2][0]:
                #    (v, s) = r[1]
                #elif r[0][0] == r[2][0]:
                #    (v, s) = r[0]
                #else:
                #    (v, s) = (None, None)
                    
                (v, s) = get_answer(llm, messages, table, q)    
                
                rng = 0.01
                if  not isinstance(v, tuple) and not isinstance(v, str)  and ( v == _q['answer'] or (v > _q['answer']-rng and v < _q['answer']+rng)):
                  print("\033[92m Success: " + str(v)+'\033[0m')
                else:    
                    print("\033[91m failure: " + str(v), 'good answer: ', _q['answer'],'\033[0m' ) 
                
                
                (pred_value, pred_scale) = (v,s)
                if isinstance(pred_value, tuple) and len(pred_value) == 2:
                    print('$$$$')
                    (pred_value, pred_scale) = pred_value
                if pred_scale == "%" or pred_scale == "percentage"  :
                    pred_scale = 'percent'
                if pred_scale not in ["", 'thousand', 'million', 'billion', 'percent']:
                    print('Invalid ', pred_scale)
                    pred_scale = ""    
                #if item['table']['uid'] in res_scale and pred_scale != 'percent':
                #    pred_scale = res_scale[item['table']['uid']]
                err=""
                if isinstance(pred_value, str):
                    #print("string")
                    if  pred_value.startswith('[Error]'):
                        (pred_value, pred_scale) = ("", "")
                        err = v                                                        
                res.append(({"answer_type":_q["answer_type"], "answer": _q["answer"], 'scale': _q["scale"]}, pred_value, pred_scale, _q, code, "",value_list, err))
    except Exception as e:
        s = '[Outer Exception]'+ str(e)
        print(s)            

0it [00:00, ?it/s]

b2786c1a-37de-4120-b03c-32bf5c81f157
What was the change in the amount for Appliances in 2019 from 2018? Success: -94.0
fe11f001-3bfe-4089-8108-412676f0a780
What was the percentage change in the amount for Appliances in 2019 from 2018? Success: -12.14
5103aed0-b4e8-4fae-bf78-e2c9f4ba84cf
What is the difference between the domestic and international discount rates as at September 30, 2019? Success: 2.1
4dc8be43-d8d9-4b08-9ffd-9c19012361ce
What is the year on year percentage change in domestic discount rate between 2018 and 2019? Success: 6.67
6c44a1a8-0785-43a0-90ab-7e21df2c57d9
What is the year on year percentage change in international expected return on plan assets between 2018 and 2019? Success: -8.11
a0414f81-8dc2-44b2-a441-2c9d9c805c4d
What is the 2019 average defined contribution schemes? Success: 172.0
bf7abd62-d9cd-48d2-8826-1457684019a3
What is the 2019 average defined benefit schemes? Success: 50.5
4d259081-6da6-44bd-8830-e4de0031744c
What is the difference between 2019 avera

In [26]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_, _,_,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=False)
print( pred_em, pred_f1, scale_score)

0.5967741935483871 0.5967741935483871 0.6975806451612904


In [27]:
good = 0
table_only = 0
for ans, pred, pred_scale, q,code, _,value_list,err in res:
    llimit = ans['answer']*0.99
    ulimit = ans['answer']*1.01
    #if q['uid'] == '263d03ec-83d2-48df-8376-1a72167798f7':
    #    print(q['uid'], '{',pred, '}', ans['answer'], ans['scale'], err)
    if q['answer_from'] == 'table':
        table_only = table_only + 1
    else:
        continue
    if pred  is None or isinstance(pred, tuple) or isinstance(pred, str):
        #if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
       
        #type error
        #if isinstance(pred, str):
        #    print (pred, pred_scale, ' -- ' , ans['answer'])
        #if isinstance(pred, tuple):
        #    print (pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
        #print('===============================================================================')
        print(q['uid'], '|', q['question'],'|', q['derivation'], '|',err, '||', ans['answer'], '|', ans['scale'])
        continue
    
    #pred = round(pred, 2)
    
    if (pred > 0 and llimit < pred and pred < ulimit) or (pred < 0 and llimit > pred and pred > ulimit) or pred==ulimit or pred == llimit:
        good = good + 1
        #if ans['scale'] != pred_scale:
        #    print(q['question'])
        #    print(print ('SS',q['uid'],q['derivation'], pred, '[', pred_scale, '] <S> ' , ans['answer'], ans['scale']))
            
        #if ans['scale'] == pred_scale and  pred_scale == 'thousand':
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' == ' , ans['answer'], ans['scale'])
        
        #if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
        #if  ans['scale'] == 'thousand':
        #    print (q['uid'], pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
          
    else:
        #value error
        #  if ans['scale'] != pred_scale:
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
        #if ans['answer'] != pred and ans['scale'] == pred_scale and pred_scale != "":
        #    print ('***********', q['uid'], 'pred: ',pred, 'scale:',pred_scale, ' <> ' , ans['answer'], ans['scale'])
       
        #None
        print (q['uid'],'|', q['question'],'|', q['derivation'], '|',pred, '|',pred_scale, '|', ans['answer'], '|', ans['scale'])
        #print(q['question'])
        #print(value_list)
        #print(value_list)
        #print(code)
        
    #if isinstance(pred, str):
    #    print (pred, pred_scale, ' -- ' , ans['answer'])
    #if isinstance(pred, tuple):
    #    print (pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
        
#print (good/len(res))
print (good/table_only)

a983501d-2eec-486d-9661-e520c7c8af5e | What was the average difference between EBITDA and underlying EBITDA for both FYs? | ((85,123 - 79,046) + (63,954 - 62,575)) / 2  | 2349.0 |  | 3728 | thousand
64c902c6-f426-4432-84b3-c10b3065716f | What was the difference in net profit between both FYs? | -9,819 - 6,639  | 16458.0 |  | -16458 | thousand
010df393-a62e-408d-888a-045a4e435e6a | What is the difference in amount between Deferred Revenue and Other non-current liabilities as reported? | (38,476-24,264) | -14212.0 | thousand | 14212 | thousand
5c8c999e-354f-4693-9b2d-29e3c03cb2af | What was the change in the preferred stock disposition from 2018 to 2019? | -9.9 - 0 | [Error] || -9.9 | percent
94ef7822-a201-493e-b557-a640f4ea4d83 | What is the change in the average total current tax expense between 2017-2018, and 2018-2019? | [(1,062+1,018)/2] - [(1,018+950)/2] | 34.0 | million | 56 | million
73693527-ed4b-4d07-941e-0e654095a43d | What was the employee termination costs as a proportion of

In [29]:
len(res), table_only

(496, 496)

In [72]:
good = 0
for ans, pred, pred_scale, q,code, _,value_list,err in res:
    llimit = ans['answer']*0.99
    ulimit = ans['answer']*1.01
    #if q['uid'] == '263d03ec-83d2-48df-8376-1a72167798f7':
    print(q['uid'], '{',pred_scale, '}', ans['scale'], ans['answer'], err, end='')
    if pred_scale == ans['scale']:
        print()
    else:
        print('<------------ XXX')
    

eb787966-fa02-401f-bfaf-ccabf3828b23 {  } million -12.6 <------------ XXX
05b670d3-5b19-438c-873f-9bf6de29c69e { percent } percent -22.22 
b2786c1a-37de-4120-b03c-32bf5c81f157 { million } million -94 
fe11f001-3bfe-4089-8108-412676f0a780 { percent } percent -12.14 
5103aed0-b4e8-4fae-bf78-e2c9f4ba84cf { percent } percent 2.1 
4dc8be43-d8d9-4b08-9ffd-9c19012361ce { percent } percent 6.67 
6c44a1a8-0785-43a0-90ab-7e21df2c57d9 { percent } percent -8.11 
a0414f81-8dc2-44b2-a441-2c9d9c805c4d { million } million 172 
bf7abd62-d9cd-48d2-8826-1457684019a3 { million } million 50.5 
4d259081-6da6-44bd-8830-e4de0031744c { million } million 121.5 
bed1fce2-69cb-4d1e-a34a-01950a1770bd { percent } percent 26.82 
348d031d-73ab-4b35-af46-998cfef25775 { percent } percent 25.1 
aea850af-68cb-4cc1-80b2-c9c2860b273e { percent } percent 2.98 
ba6783f3-8207-419a-b407-3f688682caef {  }  2.93 
263d03ec-83d2-48df-8376-1a72167798f7 {  }  0.87 
78fc6d55-c20c-4f71-99fe-bc40a16e61d0 {  }  -361 
35d602ae-9131-4291-

In [21]:
good = 0
for ans, pred, pred_scale, q,code, _,value_list,err in res:
    llimit = ans['answer']*0.99
    ulimit = ans['answer']*1.01
    #if q['uid'] == '263d03ec-83d2-48df-8376-1a72167798f7':
    #    print(q['uid'], '{',pred, '}', ans['answer'], ans['scale'], err)
    if (isinstance(pred, float) and pred/1000000 ==  ans['answer'])  and pred != 0 :
        print(pred,ans['answer'])

In [28]:
good = 0
for ans, pred, pred_scale, q,code, _,value_list,err in res:
    llimit = ans['answer']*0.99
    ulimit = ans['answer']*1.01
    #if q['uid'] == '263d03ec-83d2-48df-8376-1a72167798f7':
    #    print(q['uid'], '{',pred, '}', ans['answer'], ans['scale'], err)
    if pred ==  ans['answer']*-1 and pred != 0:
        print(pred,ans['answer'])
        #if ans['scale'] != pred_scale:

16458.0 -16458
-14212.0 14212
2.0 -2
-1226114.0 1226114
-29.84 29.84
-103.67 103.67
-112.0 112
6850.0 -6850
-1.9 1.9
-153391.0 153391
32074.0 -32074
6.1 -6.1
52.63 -52.63
-185.3 185.3
-11888.0 11888
31.58 -31.58
-236.0 236
-1503.0 1503
-15.5 15.5
-368.13 368.13
